### Train and validation setup for transformer models

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt


from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

import timm
import torch
import torch.nn as nn
from torch.utils.data.sampler import WeightedRandomSampler

from tqdm import tqdm


from src.dl_model_utils import get_transform_train,get_transform_valid, Cholec80Dataset, EarlyStopping, start_timer,end_timer_and_print
from src.models import  *


In [2]:
INSTRUMENT_MAP = {
    '':0,
    'tool_Grasper':1,
    'tool_Grasper, tool_Hook':2,
    'tool_Hook':3,
    'tool_Grasper, tool_Irrigator':4,
    'tool_Irrigator':5,
    'tool_Bipolar':6,
    'tool_Grasper, tool_Bipolar':7,
    'tool_Grasper, tool_Clipper':8,
    'tool_Clipper':9,
    'tool_Grasper, tool_Scissors':10,
    'tool_SpecimenBag':11,
    'tool_Grasper, tool_SpecimenBag':12,
    'tool_Scissors':13,
    'tool_Grasper, tool_Bipolar, tool_Irrigator':14,
    'tool_Bipolar, tool_Irrigator':15,
    'tool_Bipolar, tool_SpecimenBag':16,
    'tool_Grasper, tool_Bipolar, tool_SpecimenBag':17,
    'tool_Irrigator, tool_SpecimenBag':18,
    'tool_Bipolar, tool_Irrigator, tool_SpecimenBag':19,
    'tool_Grasper, tool_Irrigator, tool_SpecimenBag':20,
    'tool_Grasper, tool_Clipper, tool_Irrigator':21,
    'tool_Clipper, tool_Irrigator':22,
    'tool_Grasper, tool_Clipper, tool_SpecimenBag':23,
    'tool_Grasper, tool_Scissors, tool_SpecimenBag':24,
    'tool_Hook, tool_Irrigator':25,
    'tool_Grasper, tool_Hook, tool_Irrigator':26,
    'tool_Bipolar, tool_Scissors':27
    }

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device Type : {}".format(device))

Device Type : cuda


In [4]:
#Import train and validation sets
# train_df = pd.read_parquet('data/train_df.parquet')
# val_df = pd.read_parquet('data/val_df.parquet')
# train_df = pd.read_parquet('data/ordered_train_df.parquet')
# val_df = pd.read_parquet('data/ordered_val_df.parquet')
train_df = pd.read_parquet('data/ord_train_df.parquet')
val_df = pd.read_parquet('data/ord_val_df.parquet')


#Sample train and val datasets at 1fps
train_df = train_df[train_df['frame'] % 25 == 0].reset_index(drop=True)
val_df = val_df[val_df['frame'] % 25 == 0].reset_index(drop=True)

In [11]:
train_df['tool_target'] = train_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].dot(train_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].columns + ', ').str.rstrip(', ').map(INSTRUMENT_MAP).values
val_df['tool_target'] = val_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].dot(val_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].columns + ', ').str.rstrip(', ').map(INSTRUMENT_MAP).values

TypeError: can't multiply sequence by non-int of type 'float'

In [6]:
#Get train label weights and convert to torch tensor
tool_weights=class_weight.compute_class_weight('balanced',classes=np.unique(train_df['tool_target']),y=train_df['tool_target'].to_numpy())
tool_weights=torch.tensor(tool_weights,dtype=torch.float)
print(f'Weighted classes: {tool_weights}')

Weighted classes: tensor([3.5000e-01, 2.6292e-01, 9.2914e-02, 2.0479e-01, 1.4761e+00, 2.3115e+00,
        2.8420e+00, 1.9868e+00, 1.4197e+00, 2.9995e+00, 3.1497e+00, 4.3800e+00,
        8.7081e-01, 5.6997e+00, 6.1303e+00, 1.3465e+01, 2.4280e+01, 9.0960e+00,
        1.0560e+01, 1.5813e+01, 5.7852e+00, 2.5696e+02, 7.7088e+02, 3.0835e+03,
        1.1013e+02, 5.4097e+01, 2.2673e+01, 4.0573e+01])


In [12]:
def train_validate_tool_detect(train_df, val_df, method=0, device='cuda',epochs=100, model_num=0, optimizer_num = 1,batch_size = 128, learning_rate = 0.0005, weight_decay=0.05,momentum=0.9, patience=30, patience_delta = 0.005,img_in_size = 224, classes=7):
    '''
    Model nums:
    Transformers tool detection
    Model 0: ViT
    Model 1 & 2: DeiT Distilled & DeiT non distilled huge
    '''
    
    #Get train label weights and convert to torch tensor
    tool_weights=class_weight.compute_class_weight('balanced',classes=np.unique(train_df['tool_target']),y=train_df['tool_target'].to_numpy())
    tool_weights=torch.tensor(tool_weights,dtype=torch.float)
    print(f'Weighted classes: {tool_weights}')

    #Create datasets for phase segmentation
    trainset = Cholec80Dataset(train_df,  get_transform_train(img_in_size))
    validationset = Cholec80Dataset(val_df,  get_transform_valid(img_in_size))

    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True
    )
    validationloader = torch.utils.data.DataLoader(
        validationset, 
        batch_size=batch_size, 
        shuffle=False
    )

    #Initialise tool detection loss
    criterion = nn.BCEWithLogitsLoss(reduction='sum')
    criterion_tool = nn.CrossEntropyLoss(weight=tool_weights)
    if device == 'cuda':
        criterion.to(device)
        criterion_tool.to(device)
        
    #Initialize early stopping
    early_stopping = EarlyStopping(patience, patience_delta)


    #Model selection
    if model_num == 0:
        model = ViT_OD()
        model_name = 'ViT_ord_od'
        model.freeze()
    elif model_num == 1:
        model = DeiT_Distilled_OD(num_classes=classes)
        model_name = 'DeiT_ord_od'
        model.freeze()
    # elif model_num == 2:
    #     model = DeiT_OD_phase()
    #     model_name = 'DeiT3_ord_od'
    #     model.freeze()

    #Final calculated learning rate
    learning_rate_final = learning_rate * batch_size / 128

    #Select optimizer for job
    optimizer = None
    if optimizer_num == 0:
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_num == 1:
        # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
        optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, weight_decay=weight_decay)

    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=20)

    if device == 'cuda':
        model.to(device)

    start_timer()
    for epoch in range(0,epochs):
        print(f'Training Epoch: {epoch+1} of {epochs}..')

        train_loss = 0.0
        train_correct = 0
        total = 0.0
        batch_gt = []
        batch_pred = []

        # Outputs for HMM
        img_path = []
        true_labels = []
        softmax_out = []
        predicted_labels = []


        model.train()
        for i, (train_inputs,train_img_id, train_labels,train_tools,train_tool_targets) in tqdm(enumerate(trainloader, 0)):
            img_path = img_path + list(train_img_id)
            true_labels = true_labels + train_tool_targets.tolist()

            if device == 'cuda':
                train_inputs, train_labels, train_tools,train_tool_targets = train_inputs.to(device), train_labels.to(device), train_tools.to(device), train_tool_targets.to(device)

            optimizer.zero_grad()

            train_outputs, sigmoid_outputs  = model.forward(train_inputs)
            if method ==0:
                train_preds = (sigmoid_outputs > 0.5).float()
            else:
                _, train_preds = torch.max(train_outputs.data, 1)

            if method == 0:
                loss = criterion(train_outputs, train_tools)
            else:
                loss = criterion_tool(train_outputs, train_tool_targets)
                
            loss.backward()
            optimizer.step()

            softmax_out = softmax_out + train_outputs.cpu().detach().tolist()
            predicted_labels = predicted_labels + train_preds.cpu().tolist()

            train_loss += loss.item()
            if method == 0:
                batch_gt.append(np.concatenate(train_tools.cpu().tolist()))
                batch_pred.append(np.concatenate(train_preds.cpu().tolist()))
            else:
                total += train_labels.size(0)
                train_correct += train_preds.eq(train_tool_targets).sum().item()
                
        if method == 0:
            all_batch_gt = np.concatenate(batch_gt)
            all_batch_pred = np.concatenate(batch_pred)

        hmm_df = pd.DataFrame(
            {'img_path': img_path,
                'true_labels': true_labels,
                'cnn_output': softmax_out,
                'predicted_labels': predicted_labels
                })
        if method == 0:
            print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%%'
            % (train_loss / len(trainloader),100. * accuracy_score(all_batch_gt,all_batch_pred)))
        else:
            print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%% (%d/%d)'
            % (train_loss / len(trainloader), 100. * train_correct / total, train_correct, total))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0.0
        val_batch_gt = []
        val_batch_pred = []
            
        with torch.no_grad():
            for j, (val_inputs,val_img_id, val_labels,val_tools,val_tool_targets) in enumerate(validationloader):
                if device == 'cuda':
                    val_inputs, val_labels, val_tools, val_tool_targets = val_inputs.to(device), val_labels.to(device), val_tools.to(device), val_tool_targets.to(device)

                val_outputs, val_sigmoid_outputs = model.forward(val_inputs)
                if method == 0:
                    val_preds = (val_sigmoid_outputs > 0.5).float()
                else:
                    _, val_preds = torch.max(val_outputs.data, 1)

                if method == 0:
                    loss = criterion(val_outputs, val_tools)
                else:
                    loss = criterion_tool(val_outputs, val_tool_targets)

                val_loss += loss.item()
                if method == 0:
                    val_batch_gt.append(np.concatenate(val_tools.cpu().tolist()))
                    val_batch_pred.append(np.concatenate(val_preds.cpu().tolist()))
                else:
                    val_total += val_labels.size(0)
                    val_correct += val_preds.eq(val_tool_targets).sum().item()

            if method == 0:
                all_batch_gt = np.concatenate(val_batch_gt)
                all_batch_pred = np.concatenate(val_batch_pred)

            if method == 0:
                print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%%'
                % (val_loss / len(validationloader), 100. * accuracy_score(all_batch_gt,all_batch_pred)))
            else:
                print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%% (%d/%d)'
                % (val_loss / len(validationloader), 100. * val_correct / val_total, val_correct, val_total))
    
            scheduler.step(val_loss)
            #Get average for early stopping check and call the early stopping class to perform calc
            val_loss_avg = val_loss/len(validationloader)
            early_stopping(val_loss_avg, model, model_name, hmm_df)

            #If limit reached stop execution
            if early_stopping.stop:
                break

    end_timer_and_print()

In [ ]:
start_time = None
train_validate_tool_detect(train_df, val_df,method=0, model_num=1,img_in_size=224)

In [5]:
def model_train_validate_transformer(train_df, val_df,device='cuda',epochs=100, model_num=0, optimizer_num = 1,batch_size = 128, learning_rate = 0.0005, weight_decay=0.05,momentum=0.9, patience=30, patience_delta = 0.0005,img_in_size = 224):
    '''
    Model nums:
    Transformers
    Model 0 & 10: ViT ViT LSTM
    Model 1 & 4: DeiT Distilled & DeiT non distilled huge
    Model 2: BeiT
    Model 3: Cross_ViT
    Nodel 5: Swin
    '''

    #Get train label weights and convert to torch tensor
    phase_weights=class_weight.compute_class_weight('balanced',classes=np.unique(train_df['phase']),y=train_df['phase'].to_numpy())
    phase_weights=torch.tensor(phase_weights,dtype=torch.float)
    print(f'Weighted classes: {phase_weights}')


    #Create datasets for phase segmentation
    trainset = Cholec80Dataset(train_df,  get_transform_train(img_in_size))
    validationset = Cholec80Dataset(val_df,  get_transform_valid(img_in_size))
    if model_num >= 10:
        trainloader = torch.utils.data.DataLoader(
            trainset,
            batch_size=batch_size,
            shuffle=False
        )
    else:
        trainloader = torch.utils.data.DataLoader(
            trainset,
            batch_size=batch_size,
            shuffle=True
        )
    validationloader = torch.utils.data.DataLoader(
        validationset, 
        batch_size=batch_size, 
        shuffle=False
    )

    #Initialise two required loss functions
    criterion = nn.CrossEntropyLoss(weight=phase_weights)
    if device == 'cuda':
        criterion.to(device)
        
    #Initialize early stopping
    early_stopping = EarlyStopping(patience, patience_delta)


    #Model selection
    if model_num == 0:
        model = ViT()
        model_name = 'ViT_ord'
        model.freeze()
    elif model_num == 1:
        model = DeiT_Distilled()
        model_name = 'DeiT_ord_final_flr'
        model.freeze()
    elif model_num == 2:
        model = BEIT()
        model_name = 'BEIT_ord'
        model.freeze()
    elif model_num == 3:
        model = Cross_Vit()
        model_name = 'Cross_ViT_ord'
        model.freeze()
    elif model_num == 4:
        model = DeiT()
        model_name = 'DeiT3_ord'
        model.freeze()
    elif model_num == 5:
        model = DeiT_Distilled()
        model_name = 'DeiT_ord_32'
        model.freeze()     
    elif model_num == 10:
        model = ViT_LSTM()
        model_name = 'ViT_LSTM_ord'
        model.freeze()
    elif model_num == 11:
        model = DeiT_Distilled_LSTM()
        model_name = 'DeiT_LSTM_ord'
        model.freeze()
    elif model_num == 12:
        model = DeiT_Distilled_LSTM(dataset=1)
        model_name = 'DeiT_LSTM_ord_32'
        model.freeze()
    # else:
    #     model = Swin()
    #     model_name = 'Swin_ord'
    #     model.freeze()

    #Final calculated learning rate
    learning_rate_final = learning_rate# * batch_size / 512

    #Select optimizer for job
    optimizer = None
    if optimizer_num == 0:
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_num == 1:
        # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
        optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=20, verbose=True)

    if device == 'cuda':
        model.to(device)

    start_timer()
    for epoch in range(0,epochs):
        print(f'Training Epoch: {epoch+1} of {epochs}..')

        train_loss = 0.0
        train_correct = 0
        total = 0.0

        # Outputs for HMM
        img_path = []
        true_labels = []
        soft_max_out = []
        predicted_labels = []


        model.train()
        for i, (train_inputs,train_img_id, train_labels,train_tools) in tqdm(enumerate(trainloader, 0)):
            img_path = img_path + list(train_img_id)
            true_labels = true_labels + train_labels.tolist()

            if device == 'cuda':
                train_inputs, train_labels, train_tools = train_inputs.to(device), train_labels.to(device), train_tools.to(device)

            optimizer.zero_grad()

            train_outputs  = model.forward(train_inputs)
            
            _, train_preds = torch.max(train_outputs.data, 1)
            loss = criterion(train_outputs, train_labels)

            loss.backward()
            optimizer.step()

            soft_max_out = soft_max_out + train_outputs.cpu().detach().tolist()
            predicted_labels = predicted_labels + train_preds.cpu().tolist()

            train_loss += loss.item()
            total += train_labels.size(0)
            train_correct += train_preds.eq(train_labels).sum().item()

        hmm_df = pd.DataFrame(
            {'img_path': img_path,
                'true_labels': true_labels,
                'cnn_output': soft_max_out,
                'predicted_labels': predicted_labels
                })
        
        print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%% (%d/%d)'
        % (train_loss / len(trainloader), 100. * train_correct / total, train_correct, total))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0.0
            
        with torch.no_grad():
            for j, (val_inputs,val_img_id, val_labels,val_tools) in enumerate(validationloader):
                if device == 'cuda':
                    val_inputs, val_labels, val_tools = val_inputs.to(device), val_labels.to(device), val_tools.to(device)

                val_outputs = model.forward(val_inputs)
    
                _, val_preds = torch.max(val_outputs.data, 1)
                loss = criterion(val_outputs, val_labels)




                val_loss += loss.item()
                val_total += val_labels.size(0)
                val_correct += val_preds.eq(val_labels).sum().item()

        
            print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%% (%d/%d)'
                % (val_loss / len(validationloader), 100. * val_correct / val_total, val_correct, val_total))
    
            # scheduler.step(val_loss)
            #Get average for early stopping check and call the early stopping class to perform calc
            val_loss_avg = val_loss/len(validationloader)
            early_stopping(val_loss_avg, model, model_name, hmm_df)

            #If limit reached stop execution
            if early_stopping.stop:
                break

    end_timer_and_print()

In [6]:
start_time = None
model_train_validate_transformer(train_df, val_df,optimizer_num=0, model_num=12,batch_size=256, learning_rate=1e-4)

Weighted classes: tensor([3.2690, 0.3413, 1.6529, 0.5062, 3.2902, 1.6971, 3.4443])


c:\Users\vkula\anaconda3\lib\site-packages\torch\cuda\memory.py:278: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Training Epoch: 1 of 100..


278it [05:35,  1.21s/it]


278 Training Loss Phase: 1.56493 | Training Acc Phase: 46.16067% (32787/71028)
60 Validation Loss Phase: 1.30362 | Validation Acc Phase: 60.95618% (9333/15311)
Training Epoch: 2 of 100..


...

278 Training Loss Phase: 0.17229 | Training Acc Phase: 95.19908% (67618/71028)
60 Validation Loss Phase: 0.73855 | Validation Acc Phase: 80.22337% (12283/15311)
Early Stopping count is at: 31 maximum is: 30
Training Epoch: 99 of 100..


278it [04:25,  1.05it/s]


278 Training Loss Phase: 0.21777 | Training Acc Phase: 93.51523% (66422/71028)
60 Validation Loss Phase: 0.57884 | Validation Acc Phase: 79.98824% (12247/15311)
Early Stopping count is at: 32 maximum is: 30
Training Epoch: 100 of 100..


278it [04:26,  1.04it/s]


278 Training Loss Phase: 0.32541 | Training Acc Phase: 89.17188% (63337/71028)
60 Validation Loss Phase: 0.65893 | Validation Acc Phase: 81.72556% (12513/15311)
Early Stopping count is at: 33 maximum is: 30
Total execution time = 32704.768 sec
Max memory used by tensors = 2592783872 bytes
